<a href="https://colab.research.google.com/github/GonorAndres/Analisis_Numerico_2025-2/blob/main/Practica2/Ejercicio21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ejercicio 21

Sean $A$ y $b$ dados como sigue:

#### (a)

$$
A =
\begin{bmatrix}
4 & -1 & 3 \\
-8 & 4 & -7 \\
12 & 1 & 8
\end{bmatrix}, \quad
b =
\begin{bmatrix}
-8 \\
19 \\
-19
\end{bmatrix}
$$

#### (b)

$$
A =
\begin{bmatrix}
1 & 4 & -2 & 1 \\
-2 & -4 & -3 & 1 \\
1 & 16 & -17 & 9 \\
2 & 4 & -9 & -3
\end{bmatrix}, \quad
b =
\begin{bmatrix}
3.5 \\
-2.5 \\
15 \\
10.5
\end{bmatrix}
$$

#### (c)

$$
A =
\begin{bmatrix}
4 & 5 & -1 & 2 & 3 \\
12 & 13 & 0 & 10 & 3 \\
-8 & -8 & 5 & -11 & 4 \\
16 & 18 & -7 & 20 & 4 \\
-4 & -9 & 31 & -31 & -1
\end{bmatrix}, \quad
b =
\begin{bmatrix}
34 \\
93 \\
-33 \\
131 \\
-58
\end{bmatrix}
$$

Resolver cada sistema $Ax = b$ con las rutinas:

- **Factorización LU**
- **Factorización LU con pivoteo parcial**
- **Factorización LU con pivoteo total**

Comparar los resultados.


In [2]:
#Funciones de la clase

import numpy as np
from numpy import linalg as LA


def SustitucionDelante(Mat, b):
    """
    Realiza la sustitución hacia adelante para resolver un sistema de ecuaciones lineales
    representado por una matriz triangular inferior.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Una matriz triangular inferior de tamaño n x n.
    b : numpy.ndarray
        Un vector de términos independientes de tamaño n.

    Retorna:
    --------
    x : numpy.ndarray
        Un vector solución de tamaño n que satisface la ecuación Mat @ x = b.

    Descripción:
    ------------
    Esta función resuelve un sistema de ecuaciones lineales de la forma Mat @ x = b,
    donde Mat es una matriz triangular inferior. Utiliza el método de sustitución hacia adelante,
    comenzando desde la primera fila de la matriz y avanzando hacia la última.

    Ejemplo:
    --------
    >>> Mat = np.array([[1, 0, 0],
    ...                 [2, 3, 0],
    ...                 [4, 5, 6]])
    >>> b = np.array([1, 8, 32])
    >>> SustitucionDelante(Mat, b)
    array([1., 2., 3.])
    """
    n = Mat.shape[0]
    x = np.zeros(n)

    for i in range(n):
        SumCum = 0.0
        for j in range(i):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]

    return x

def SustitucionAtras(Mat, b):
    """
    Realiza la sustitución hacia atrás para resolver un sistema de ecuaciones lineales
    representado por una matriz triangular superior.

    Parámetros:
    -----------
    Mat : numpy.ndarray
        Una matriz triangular superior de tamaño n x n.
    b : numpy.ndarray
        Un vector de términos independientes de tamaño n.

    Retorna:
    --------
    x : numpy.ndarray
        Un vector solución de tamaño n que satisface la ecuación Mat @ x = b.

    Descripción:
    ------------
    Esta función resuelve un sistema de ecuaciones lineales de la forma Mat @ x = b,
    donde Mat es una matriz triangular superior. Utiliza el método de sustitución hacia atrás,
    comenzando desde la última fila de la matriz y avanzando hacia la primera.

    Ejemplo:
    --------
    >>> Mat = np.array([[3, 2, 1],
    ...                 [0, 2, 1],
    ...                 [0, 0, 1]])
    >>> b = np.array([6, 4, 1])
    >>> SustitucionAtras(Mat, b)
    array([1., 1., 1.])
    """
    n = Mat.shape[0]
    x = np.zeros(n)

    for i in range(n-1, -1, -1):
        SumCum = 0.0
        for j in range(i+1, n):
            SumCum += Mat[i, j] * x[j]
        x[i] = (b[i] - SumCum) / Mat[i, i]

    return x

def LU(A):
  U=np.copy(A)
  L=np.eye(A.shape[0])

  for i in range(A.shape[0]):
    Li=np.eye(A.shape[0])
    for j in range(i+1,A.shape[0]):
      Li[j,i]=-U[j,i]/U[i,i]
      L[j,i]=U[j,i]/U[i,i]
    U=Li@U
  return L,U

def Solver_LU(A,b):
  L,U=LU(A)
  # El sistema que se resuelve es Ly=b
  y=SustitucionDelante(L, b)
  # El sistema que se resuelve es Ux=y
  x=SustitucionAtras(U, y)

  return x

In [3]:
def LU_parcial(A):
    n = A.shape[0]
    U = np.copy(A)
    L = np.eye(n)
    P = np.eye(n)

    for i in range(n):
        # Encontrar fila con valor absoluto máximo en la columna i
        max_val = abs(U[i, i])
        max_row = i
        for k in range(i+1, n):
            if abs(U[k, i]) > max_val:
                max_val = abs(U[k, i])
                max_row = k

        # Intercambiar filas si es necesario
        if max_row != i:
            U[[i, max_row], :] = U[[max_row, i], :] # el corchete dentro de la posición de las filas sirve como multiasignación así no creamos variables de apoyo
            P[[i, max_row], :] = P[[max_row, i], :]
            if i > 0: # tenemos que solo se intercambia después de la primera interación por justamente en la primer vuelta no hemos calculado nada y sigue siendo la identidad solo intercambiariamos ceros identicos
                L[[i, max_row], :i] = L[[max_row, i], :i] # sin embargo para la segunda vuelta ya habremos calculado por lo menos un elemento que no esta en la diagonal de la columna 1, entonces ya es necesario intercambiar
                                                          # y solo intercambiamos hasta antes de la columna i, pues solo nos interesan la parte triangular inferior, la diagonal son unos por definición
        # Eliminación gaussiana
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, :] = U[j, :] - (L[j, i] * U[i, :])

    return P, L, U

def Solver_LUParcial(A, b):
    # Resuelve el sistema Ax = b usando la factorización LU con pivoteo parcial.
    # como Ax =b y PA = LU entonces LUx = PAx = Pb así resolvemos
    # LUx = Pb
    P, L, U = LU_parcial(A)
    Pb = P @ b
    z = SustitucionDelante(L, Pb)
    x = SustitucionAtras(U, z)
    return x

In [4]:
def LU_total(A):
    n = A.shape[0]
    U = np.copy(A)
    L = np.eye(n)
    P = np.eye(n)
    Q = np.eye(n)

    for i in range(n):
        # Buscar el índice del valor absoluto máximo en submatriz U[i:, i:]
        max_val = abs(U[i, i])
        max_row, max_col = i, i
        for r in range(i, n):
            for s in range(i, n):
                if abs(U[r, s]) > max_val:
                    max_val = abs(U[r, s])
                    max_row, max_col = r, s

        # Intercambiar filas si es necesario
        if max_row != i:
            U[[i, max_row], :] = U[[max_row, i], :] # el corchete dentro de la posición de las filas sirve como multiasignación así no creamos variables de apoyo
            P[[i, max_row], :] = P[[max_row, i], :]
            if i > 0: # tenemos que solo se intercambia después de la primera interación por justamente en la primer vuelta no hemos calculado nada y sigue siendo la identidad solo intercambiariamos ceros identicos
                L[[i, max_row], :i] = L[[max_row, i], :i] # sin embargo para la segunda vuelta ya habremos calculado por lo menos un elemento que no esta en la diagonal de la columna 1, entonces ya es necesario intercambiar
                                                          # y solo intercambiamos hasta antes de la columna i, pues solo nos interesan la parte triangular inferior, la diagonal son unos por definición


        # Intercambio de columnas si es necesario
        if max_col != i:
            U[:, [i, max_col]] = U[:, [max_col, i]]
            Q[:, [i, max_col]] = Q[:, [max_col, i]]
            # Ahora vemos que no intercambiamos las columnas de L en ninguna ocasión porque L guarda en la parte triangular inferior todos los pivotes correspondientes a cada fila para la eliminación de los terminos debajo de la diagonal
            # sin embargo ahora los renglones no se movieron y el pivote de eliminación sigue correspondiendo a esa misma fila, tampoco debemos tener cuidado en el cambio de orden de las variables
            # pues el pivote afecta a toda la fila por igual no importa en que orden se encuentre y el cambio en U no afecta en nada

        # Eliminación gaussiana
        for j in range(i+1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, :] = U[j, :] - (L[j, i] * U[i, :])

    return P, L, U, Q

def Solver_LUtotal(A, b):
    # Resuelve el sistema Ax = b usando la factorización LU con pivoteo total.
    # como Ax = b y PAQ = LU entonces PA = LUQ^-1, LUQ^-1x = PAx = Pb entonces sea y = Q^-1x tenemos que LUy = Pb
    #  Resolvemos LUy = Pb
    P, L, U, Q = LU_total(A)
    Pb = P @ b
    z = SustitucionDelante(L, Pb)
    y = SustitucionAtras(U, z)
    # Resolvemos y = Q^-1x donde x = Qy
    x = Q @ y

    return x


Sean $A$ y $b$ dados como sigue:

#### (a)

$$
A =
\begin{bmatrix}
4 & -1 & 3 \\
-8 & 4 & -7 \\
12 & 1 & 8
\end{bmatrix}, \quad
b =
\begin{bmatrix}
-8 \\
19 \\
-19
\end{bmatrix}
$$

Resolver cada sistema $Ax = b$ con las rutinas:

- **Factorización LU**
- **Factorización LU con pivoteo parcial**
- **Factorización LU con pivoteo total**

Comparar los resultados.

In [5]:
# Ejercicio A
A = np.array([[ 4.0,  -1, 3],
              [ -8,  4, -7],
              [12, 1,  8]])
b = np.array([-8.0, 19, -19])

x = Solver_LU(A, b)
print("Solución x sin pivoteo =", x)

A = np.array([[ 4.0,  -1, 3],
              [ -8,  4, -7],
              [12, 1,  8]])
b = np.array([-8.0, 19, -19])

x = Solver_LUParcial(A, b)
print("Solución x con pivoteo parcial =", x)

A = np.array([[ 4.0,  -1, 3],
              [ -8,  4, -7],
              [12, 1,  8]])
b = np.array([-8.0, 19, -19])

x = Solver_LUtotal(A, b)
print("Solución x con pivoteo total =", x)




Solución x sin pivoteo = [-1.  1. -1.]
Solución x con pivoteo parcial = [-1.  1. -1.]
Solución x con pivoteo total = [-1.  1. -1.]


Sean $A$ y $b$ dados como sigue:

#### (b)

$$
A =
\begin{bmatrix}
1 & 4 & -2 & 1 \\
-2 & -4 & -3 & 1 \\
1 & 16 & -17 & 9 \\
2 & 4 & -9 & -3
\end{bmatrix}, \quad
b =
\begin{bmatrix}
3.5 \\
-2.5 \\
15 \\
10.5
\end{bmatrix}
$$


Resolver cada sistema $Ax = b$ con las rutinas:

- **Factorización LU**
- **Factorización LU con pivoteo parcial**
- **Factorización LU con pivoteo total**

Comparar los resultados.

In [6]:
## Inciso b

# Sin pivoteo
A = np.array([[  1.0,   4,  -2,   1],
              [ -2,  -4,  -3,   1],
              [  1,  16, -17,   9],
              [  2,   4,  -9,  -3]], dtype=float)

b = np.array([3.5, -2.5, 15, 10.5], dtype=float)

x = Solver_LU(A, b)
print("Solución x sin pivoteo =", x)

A = np.array([[  1.0,   4,  -2,   1],
              [ -2,  -4,  -3,   1],
              [  1,  16, -17,   9],
              [  2,   4,  -9,  -3]], dtype=float)

b = np.array([3.5, -2.5, 15, 10.5], dtype=float)


# Pivoteor Parcial
x = Solver_LUParcial(A, b)
print("Solución x con pivoteo parcial =", x)


# Pivoteo total
A = np.array([[  1.0,   4,  -2,   1],
              [ -2,  -4,  -3,   1],
              [  1,  16, -17,   9],
              [  2,   4,  -9,  -3]], dtype=float)

b = np.array([3.5, -2.5, 15, 10.5], dtype=float)

x = Solver_LUtotal(A, b)
print("Solución x con pivoteo total =", x)



Solución x sin pivoteo = [-0.5  1.  -0.5 -1. ]
Solución x con pivoteo parcial = [-0.5  1.  -0.5 -1. ]
Solución x con pivoteo total = [-0.5  1.  -0.5 -1. ]


Sean $A$ y $b$ dados como sigue:


#### (c)

$$
A =
\begin{bmatrix}
4 & 5 & -1 & 2 & 3 \\
12 & 13 & 0 & 10 & 3 \\
-8 & -8 & 5 & -11 & 4 \\
16 & 18 & -7 & 20 & 4 \\
-4 & -9 & 31 & -31 & -1
\end{bmatrix}, \quad
b =
\begin{bmatrix}
34 \\
93 \\
-33 \\
131 \\
-58
\end{bmatrix}
$$

Resolver cada sistema $Ax = b$ con las rutinas:

- **Factorización LU**
- **Factorización LU con pivoteo parcial**
- **Factorización LU con pivoteo total**

Comparar los resultados.

In [7]:
## Incisico C

# Sin Pivoteo

A = np.array([
    [  4.0,   5,  -1,   2,   3],
    [ 12,  13,   0,  10,   3],
    [ -8,  -8,   5, -11,   4],
    [ 16,  18,  -7,  20,   4],
    [ -4,  -9,  31, -31,  -1]
], dtype=float)

b = np.array([34.0, 93, -33, 131, -58], dtype=float)

x = Solver_LU(A, b)
print("Solución x sin pivoteo =", x)

# Pivoteo Parcial
A = np.array([
    [  4.0,   5,  -1,   2,   3],
    [ 12,  13,   0,  10,   3],
    [ -8,  -8,   5, -11,   4],
    [ 16,  18,  -7,  20,   4],
    [ -4,  -9,  31, -31,  -1]
], dtype=float)

b = np.array([34.0, 93, -33, 131, -58], dtype=float)

x = Solver_LUParcial(A, b)
print("Solución x con pivoteo parcial =", x)

# Pivoteo Total
A = np.array([
    [  4.0,   5,  -1,   2,   3],
    [ 12,  13,   0,  10,   3],
    [ -8,  -8,   5, -11,   4],
    [ 16,  18,  -7,  20,   4],
    [ -4,  -9,  31, -31,  -1]
], dtype=float)

b = np.array([34.0, 93, -33, 131, -58], dtype=float)

x = Solver_LUtotal(A, b)
print("Solución x con pivoteo total =", x)

Solución x sin pivoteo = [1. 2. 3. 4. 5.]
Solución x con pivoteo parcial = [1. 2. 3. 4. 5.]
Solución x con pivoteo total = [1. 2. 3. 4. 5.]


Algunos casos donde los resultados son diferentes para comprobar que los algoritmos no son incorrectos.

In [58]:
# Matriz para mostrar diferencia entre pivoteo parcial y no pivoteo
A = np.array([
    [1e-20, 1, 0],
    [1,     1, 1],
    [1,     2, 3]
], dtype=float)

b = np.array([1, 3, 6], dtype=float)

x = Solver_LU(A, b)
print("Solución x sin pivoteo =", x)

A = np.array([
    [1e-20, 1, 0],
    [1,     1, 1],
    [1,     2, 3]
], dtype=float)

b = np.array([1, 3, 6], dtype=float)

x = Solver_LUParcial(A, b)
print("Solución x con pivoteo parcial =", x)

Solución x sin pivoteo = [0. 1. 3.]
Solución x con pivoteo parcial = [1. 1. 1.]


El siguiente ejemplo fue extraído de este documento [CoarsesGrainger](https://courses.grainger.illinois.edu/cs357/su2013/lectures/lecture07.pdf), y lo agrego porque al tener todos los resultados iguales en los tres primeros incisos pensé que mis algoritmos estaban mal pero no es así solo que sirven en casos muy especificos de redondeos altos pues float trabaja con 64 bits por defecto. Y el primer ejemplo de pivoteo parcial resulta fácil de imaginar pero para el segundo de pivoteo total con el parcial no conseguía una matriz tal eso me deja en claro que el pivoteo parcial es muy poderoso

In [57]:
# ---------- Matriz diseñada para mostrar diferencia entre pivoteo parcial y total ----------
# extraido
c = 2e17 # sin embargo ci c es pequeño el resultado es identico en los tres
A = np.array([[2,  2*c],
              [1,      1]],
             dtype=float)

b = np.array([2*c,
              2],
             dtype=float)


x = Solver_LUParcial(A, b)
print("Solución x con pivoteo parcial =", x)

x2 = Solver_LUtotal(A, b)
print("Solución x2 con pivoteo total =", x2)

x3 = np.linalg.solve(A, b)
print("Solución x3 con la función de numpy =", x3)

#vamos a sustituir de vuelta para comprobar el resultado cundo c = 2e25

print("El vector original b es", b)
b1 = A@x
print("El resultado de multiplicar A con el vector x1 =", b1)

b2 = A@x2
print("El resultado de multiplicar A con el vector x2 =", b2)

b3 = A@x3
print("El resultado de multiplicar A con el vector x3 =", b3)

Solución x con pivoteo parcial = [0. 1.]
Solución x2 con pivoteo total = [1. 1.]
Solución x3 con la función de numpy = [0. 1.]
El vector original b es [4.e+17 2.e+00]
El resultado de multiplicar A con el vector x1 = [4.e+17 1.e+00]
El resultado de multiplicar A con el vector x2 = [4.e+17 2.e+00]
El resultado de multiplicar A con el vector x3 = [4.e+17 1.e+00]
